In [3]:
#Download the package
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [4]:
#install folium
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


branca-0.3.1         | 25 KB     | #############################

In [5]:
# import libraries
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
import folium # plotting library

In [10]:
#FOURSQUARE
CLIENT_ID = 'Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U' # your Foursquare ID
CLIENT_SECRET = 'DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD' # your Foursquare Secret
VERSION = '20190606'
LIMIT = 50

Our three locations being reviewed are:
- Calle de Viriato 43
- Paseo de la Castellana, 259D
- Calle de Pradillo 19

Let's get the GPS coordinates of each address:

In [106]:
#Address: Calle de Viriato 43
address_01 = 'Calle de Viriato 43, Madrid, Spain'
geolocator_01 = Nominatim(user_agent="foursquare_agent")
location_01 = geolocator_01.geocode(address_01)
latitude_01 = location_01.latitude
longitude_01 = location_01.longitude
print(latitude_01, longitude_01)

search_query_01 = 'Italian'
radius_01 = 2000
print(search_query_01 + ' .... OK!')

40.4359867 -3.7033085
Italian .... OK!


In [107]:
url_01 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_01, longitude_01, VERSION, search_query_01, radius_01, LIMIT)
url_01

'https://api.foursquare.com/v2/venues/search?client_id=Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U&client_secret=DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD&ll=40.4359867,-3.7033085&v=20190606&query=Italian&radius=2000&limit=30'

In [108]:
results_01 = requests.get(url_01).json()
results_01

{'meta': {'code': 200, 'requestId': '5d077cc2351e3d1fffa2fb3f'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/embassy_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d12c951735',
      'name': 'Embassy / Consulate',
      'pluralName': 'Embassies / Consulates',
      'primary': True,
      'shortName': 'Embassy'}],
    'hasPerk': False,
    'id': '4c0958e3ffb8c9b6bdc26961',
    'location': {'address': 'Calle de Lagasca, 98',
     'cc': 'ES',
     'city': 'Madrid',
     'country': 'España',
     'distance': 1648,
     'formattedAddress': ['Calle de Lagasca, 98',
      '28006 Madrid Madrid',
      'España'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.43260015265081,
       'lng': -3.684363892032192}],
     'lat': 40.43260015265081,
     'lng': -3.684363892032192,
     'postalCode': '28006',
     'state': 'Madrid'},
    'name': 'Embajada de la República Italiana - Ambasciata della Repubblica Italiana

In [109]:
# assign relevant part of JSON to venues
venues_01 = results_01['response']['venues']

# tranform venues into a dataframe
from pandas.io.json import json_normalize
dataframe_01 = json_normalize(venues_01)

#number of entries:
dataframe_01.shape

(22, 18)

In [111]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_01 = ['name', 'categories'] + [col for col in dataframe_01.columns if col.startswith('location.')] + ['id']
dataframe_filtered_01 = dataframe_01.loc[:, filtered_columns_01]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_01['categories'] = dataframe_filtered_01.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_01.columns = [column.split('.')[-1] for column in dataframe_filtered_01.columns]

dataframe_filtered_01

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Embajada de la República Italiana - Ambasciata...,Embassy / Consulate,"Calle de Lagasca, 98",ES,Madrid,España,NaN,1648,"[Calle de Lagasca, 98, 28006 Madrid Madrid, Es...","[{'lat': 40.43260015265081, 'lng': -3.68436389...",40.432600,-3.684364,28006,Madrid,4c0958e3ffb8c9b6bdc26961
1,iS Italian Skills,Boutique,"C. Gravina, 9",ES,Madrid,España,NaN,1515,"[C. Gravina, 9, 28004 Madrid Madrid, España]","[{'lat': 40.42306918055597, 'lng': -3.69767189...",40.423069,-3.697672,28004,Madrid,52ed42bc11d28180d18d8944
2,Mercato Italiano,Food & Drink Shop,"C. Rios Rosas, 50",ES,Madrid,España,C. Modesto Lafuente,880,"[C. Rios Rosas, 50 (C. Modesto Lafuente), 2800...","[{'lat': 40.44163816155172, 'lng': -3.69603507...",40.441638,-3.696035,28003,Madrid,584ac020ec980f4c6bc8b35f
3,italiana_madrid,Café,"Corredera Baja de San Pablo,10",ES,Madrid,España,NaN,1543,"[Corredera Baja de San Pablo,10, 28004 Madrid ...","[{'lat': 40.422138639966505, 'lng': -3.7042851...",40.422139,-3.704285,28004,Madrid,509d4c9ee4b05e06b4409edd
4,Liceo Italiano Enrico Fermi,School,NaN,ES,Madrid,España,NaN,940,"[Madrid Madrid, España]","[{'lat': 40.4419026679497, 'lng': -3.695376343...",40.441903,-3.695376,NaN,Madrid,4e310c00d1643f195b90e579
5,Camara de comercio italiana madrid,Non-Profit,"Calle Cristobal Bordiú, 54 MADRID",ES,Madrid,España,NaN,1026,"[Calle Cristobal Bordiú, 54 MADRID, 28002 Madr...","[{'lat': 40.442823749887594, 'lng': -3.6951828...",40.442824,-3.695183,28002,Madrid,519dcc9d454a4051f5866413
6,Cámara de Comercio e Industria Italiana para E...,Non-Profit,"Calle Cristobal Bordiú, 54",ES,Madrid,España,NaN,1017,"[Calle Cristobal Bordiú, 54, 28002 Madrid Madr...","[{'lat': 40.44274209763634, 'lng': -3.69521498...",40.442742,-3.695215,28002,Madrid,519dcef6454a4051f588105c
7,Capperi Mercato Italiano,Deli / Bodega,C/ Fernando VI 2,ES,Madrid,España,NaN,1238,"[C/ Fernando VI 2, Madrid Madrid, España]","[{'lat': 40.42590349266851, 'lng': -3.69712527...",40.425903,-3.697125,NaN,Madrid,4b62d14bf964a520ef532ae3
8,Matteo Cucina Italiana,Italian Restaurant,Calle Ayala 28,ES,Madrid,España,NaN,1792,"[Calle Ayala 28, 28001 Madrid Madrid, España]","[{'lat': 40.427054401800795, 'lng': -3.6857014...",40.427054,-3.685701,28001,Madrid,532d7469498e6f7a11b53ddc
9,Instituto Italiano de Cultura,General College & University,"Calle Mayor, 86",ES,Madrid,España,NaN,2452,"[Calle Mayor, 86, 28013 Madrid Madrid, España]","[{'lat': 40.41519676638148, 'lng': -3.71288344...",40.415197,-3.712883,28013,Madrid,4be83d5b9a54a593a8b70911


In [112]:
dataframe_filtered_01.name

0     Embajada de la República Italiana - Ambasciata...
1                                     iS Italian Skills
2                                      Mercato Italiano
3                                       italiana_madrid
4                           Liceo Italiano Enrico Fermi
5                    Camara de comercio italiana madrid
6     Cámara de Comercio e Industria Italiana para E...
7                              Capperi Mercato Italiano
8                                Matteo Cucina Italiana
9                         Instituto Italiano de Cultura
10                                      Cucina italiana
11                       Aperitivo Italiano en Malasaña
12                          Bellamia Gelateria Italiana
13                              4D Caffetteria Italiana
14                             Heladeria Salon Italiano
15                                   Il Centro Italiano
16                                        Café italiano
17                          Pizzeria Italiana Ma

In [113]:
venues_map_01 = folium.Map(location=[latitude_01, longitude_01], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent
folium.features.CircleMarker(
    [latitude_01, longitude_01],
    radius=10,
    color='red',
    popup='Our venue spot',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_01)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_01.lat, dataframe_filtered_01.lng, dataframe_filtered_01.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_01)

# display map
venues_map_01

In [119]:
#Address: Paseo de la Castellana, 259D
address_02 = 'Paseo de la Castellana, 259D, Madrid, Spain'
geolocator_02 = Nominatim(user_agent="foursquare_agent")
location_02 = geolocator_02.geocode(address_02)
latitude_02 = location_02.latitude
longitude_02 = location_02.longitude
print(latitude_02, longitude_02)

search_query_02 = 'Italian'
radius_02 = 3000
print(search_query_02 + ' .... OK!')

40.4790377 -3.6868739
Italian .... OK!


In [120]:
url_02 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_02, longitude_02, VERSION, search_query_02, radius_02, LIMIT)
url_02

'https://api.foursquare.com/v2/venues/search?client_id=Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U&client_secret=DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD&ll=40.4790377,-3.6868739&v=20190606&query=Italian&radius=3000&limit=30'

In [121]:
results_02 = requests.get(url_02).json()
results_02

{'meta': {'code': 200, 'requestId': '5d077d5ef594df1b1cee2000'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '52e81612bcbc57f1066b79f1',
      'name': 'Bistro',
      'pluralName': 'Bistros',
      'primary': True,
      'shortName': 'Bistro'}],
    'hasPerk': False,
    'id': '53e1367f498ecd25ba071919',
    'location': {'cc': 'ES',
     'city': 'Madrid',
     'country': 'España',
     'distance': 3138,
     'formattedAddress': ['Madrid Madrid', 'España'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.493719,
       'lng': -3.655233}],
     'lat': 40.493719,
     'lng': -3.655233,
     'state': 'Madrid'},
    'name': 'bistro italiano',
    'referralId': 'v-1560771934'}]}}

In [122]:
# assign relevant part of JSON to venues
venues_02 = results_02['response']['venues']

# tranform venues into a dataframe
from pandas.io.json import json_normalize
dataframe_02 = json_normalize(venues_02)

#number of entries:
dataframe_02.shape

(1, 14)

In [123]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_02 = ['name', 'categories'] + [col for col in dataframe_02.columns if col.startswith('location.')] + ['id']
dataframe_filtered_02 = dataframe_02.loc[:, filtered_columns_02]

# function that extracts the category of the venue
def get_category_type_02(row):
    try:
        categories_list_02 = row['categories']
    except:
        categories_list_02 = row['venue.categories']
        
    if len(categories_list_02) == 0:
        return None
    else:
        return categories_list_02[0]['name']

# filter the category for each row
dataframe_filtered_02['categories'] = dataframe_filtered_02.apply(get_category_type_02, axis=1)

# clean column names by keeping only last term
dataframe_filtered_02.columns = [column.split('.')[-1] for column in dataframe_filtered_02.columns]

dataframe_filtered_02

,name,categories,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,bistro italiano,Bistro,ES,Madrid,España,3138,"[Madrid Madrid, España]","[{'lat': 40.493719, 'lng': -3.655233, 'label':...",40.493719,-3.655233,Madrid,53e1367f498ecd25ba071919


In [124]:
dataframe_filtered_02.name


0    bistro italiano
Name: name, dtype: object

In [126]:
venues_map_02 = folium.Map(location=[latitude_02, longitude_02], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent
folium.features.CircleMarker(
    [latitude_02, longitude_02],
    radius=10,
    color='red',
    popup='Our venue spot',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_02)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_02.lat, dataframe_filtered_02.lng, dataframe_filtered_02.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_02)

# display map
venues_map_02

In [136]:
#Address: Calle de Pradillo 19
address_03 = 'Calle de Pradillo 19, Madrid, Spain'
geolocator_03 = Nominatim(user_agent="foursquare_agent")
location_03 = geolocator_03.geocode(address_03)
latitude_03 = location_03.latitude
longitude_03 = location_03.longitude
print(latitude_03, longitude_03)

search_query_03 = 'Italian'
radius_03 = 2000
print(search_query_03 + ' .... OK!')

40.4488336 -3.6733894
Italian .... OK!


In [137]:
url_03 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_03, longitude_03, VERSION, search_query_03, radius_03, LIMIT)
url_03

'https://api.foursquare.com/v2/venues/search?client_id=Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U&client_secret=DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD&ll=40.4488336,-3.6733894&v=20190606&query=Italian&radius=2000&limit=30'

In [138]:
results_03 = requests.get(url_03).json()
results_03

{'meta': {'code': 200, 'requestId': '5d077de99fb6b724b24fe55f'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/embassy_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d12c951735',
      'name': 'Embassy / Consulate',
      'pluralName': 'Embassies / Consulates',
      'primary': True,
      'shortName': 'Embassy'}],
    'hasPerk': False,
    'id': '4c0958e3ffb8c9b6bdc26961',
    'location': {'address': 'Calle de Lagasca, 98',
     'cc': 'ES',
     'city': 'Madrid',
     'country': 'España',
     'distance': 2032,
     'formattedAddress': ['Calle de Lagasca, 98',
      '28006 Madrid Madrid',
      'España'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.43260015265081,
       'lng': -3.684363892032192}],
     'lat': 40.43260015265081,
     'lng': -3.684363892032192,
     'postalCode': '28006',
     'state': 'Madrid'},
    'name': 'Embajada de la República Italiana - Ambasciata della Repubblica Italiana

In [139]:
# assign relevant part of JSON to venues
venues_03 = results_03['response']['venues']

# tranform venues into a dataframe
from pandas.io.json import json_normalize
dataframe_03 = json_normalize(venues_03)

#number of entries:
dataframe_03.shape

(9, 17)

In [140]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_03 = ['name', 'categories'] + [col for col in dataframe_03.columns if col.startswith('location.')] + ['id']
dataframe_filtered_03 = dataframe_03.loc[:, filtered_columns_03]

# function that extracts the category of the venue
def get_category_type_03(row):
    try:
        categories_list_03 = row['categories']
    except:
        categories_list_03 = row['venue.categories']
        
    if len(categories_list_03) == 0:
        return None
    else:
        return categories_list_03[0]['name']

# filter the category for each row
dataframe_filtered_03['categories'] = dataframe_filtered_03.apply(get_category_type_03, axis=1)

# clean column names by keeping only last term
dataframe_filtered_03.columns = [column.split('.')[-1] for column in dataframe_filtered_03.columns]

dataframe_filtered_03

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Embajada de la República Italiana - Ambasciata...,Embassy / Consulate,"Calle de Lagasca, 98",ES,Madrid,España,NaN,2032,"[Calle de Lagasca, 98, 28006 Madrid Madrid, Es...","[{'lat': 40.43260015265081, 'lng': -3.68436389...",40.432600,-3.684364,28006,Madrid,4c0958e3ffb8c9b6bdc26961
1,Mercato Italiano,Food & Drink Shop,"C. Rios Rosas, 50",ES,Madrid,España,C. Modesto Lafuente,2078,"[C. Rios Rosas, 50 (C. Modesto Lafuente), 2800...","[{'lat': 40.44163816155172, 'lng': -3.69603507...",40.441638,-3.696035,28003,Madrid,584ac020ec980f4c6bc8b35f
2,Il Centro Italiano,Student Center,General diaz porlier,ES,Madrid,España,NaN,1222,"[General diaz porlier, Madrid Madrid, España]","[{'lat': 40.43893741301658, 'lng': -3.67963863...",40.438937,-3.679639,NaN,Madrid,4d405eeeadb6236aaac58740
3,Consolato Italiano a Madrid,Government Building,Agustin De Betancourt 3,ES,Madrid,España,NaN,1498,"[Agustin De Betancourt 3, Madrid Madrid, España]","[{'lat': 40.43546789183154, 'lng': -3.67546849...",40.435468,-3.675468,NaN,Madrid,511e24e6e4b0f3711007ae7f
4,Il Centro Italiano,Student Center,General diaz porlier,ES,Madrid,España,NaN,1588,"[General diaz porlier, Madrid Madrid, España]","[{'lat': 40.43474589197704, 'lng': -3.67636489...",40.434746,-3.676365,NaN,Madrid,4d405eee915a3704c1d4147f
5,Camara de comercio italiana madrid,Non-Profit,"Calle Cristobal Bordiú, 54 MADRID",ES,Madrid,España,NaN,1963,"[Calle Cristobal Bordiú, 54 MADRID, 28002 Madr...","[{'lat': 40.442823749887594, 'lng': -3.6951828...",40.442824,-3.695183,28002,Madrid,519dcc9d454a4051f5866413
6,Cámara de Comercio e Industria Italiana para E...,Non-Profit,"Calle Cristobal Bordiú, 54",ES,Madrid,España,NaN,1969,"[Calle Cristobal Bordiú, 54, 28002 Madrid Madr...","[{'lat': 40.44274209763634, 'lng': -3.69521498...",40.442742,-3.695215,28002,Madrid,519dcef6454a4051f588105c
7,Liceo Italiano Enrico Fermi,School,NaN,ES,Madrid,España,NaN,2016,"[Madrid Madrid, España]","[{'lat': 40.4419026679497, 'lng': -3.695376343...",40.441903,-3.695376,NaN,Madrid,4e310c00d1643f195b90e579
8,Café italiano,Internet Cafe,NaN,ES,Madrid,España,NaN,2386,"[Madrid Madrid, España]","[{'lat': 40.451143, 'lng': -3.701401, 'label':...",40.451143,-3.701401,NaN,Madrid,539a95a1498eda0a5f877242


In [141]:
dataframe_filtered_03.name


0    Embajada de la República Italiana - Ambasciata...
1                                     Mercato Italiano
2                                   Il Centro Italiano
3                          Consolato Italiano a Madrid
4                                   Il Centro Italiano
5                   Camara de comercio italiana madrid
6    Cámara de Comercio e Industria Italiana para E...
7                          Liceo Italiano Enrico Fermi
8                                        Café italiano
Name: name, dtype: object

In [142]:
venues_map_03 = folium.Map(location=[latitude_03, longitude_03], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent
folium.features.CircleMarker(
    [latitude_03, longitude_03],
    radius=10,
    color='red',
    popup='Our venue spot',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_03)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_03.lat, dataframe_filtered_03.lng, dataframe_filtered_03.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_03)

# display map
venues_map_03

______________
Reminding data:
______________

In [ ]:
dataframe_filtered_03.name


In [42]:
venues_map.save('Viriato.html')

In [34]:
#TRENDING LOCATIONS
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d037e256adbf5002ca6a7ec'},
 'response': {'venues': []}}

In [35]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d037e31db04f542f64a841c'},
 'response': {'venues': []}}

In [37]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [38]:
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

In [68]:
#LET'S REVIEW THE LOCATION
latitude = 40.4790377
longitude = -3.6868739
url_vir = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url_vir

'https://api.foursquare.com/v2/venues/explore?client_id=Z0J03ZIUK0D4NGFQ5BFKEWUOIZASGRALFMXLBZAAUUHMLY4U&client_secret=DH4O4SF3KKT4GQGHETVCKFUST0LU4LFC2J34KRGPZSDWBOKD&ll=40.4790377,-3.6868739&v=20190606&radius=1500&limit=30'

In [69]:
results = requests.get(url_vir).json()
'There are {} around Viriato.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Viriato.'

In [71]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4adcda33f964a520d33a21e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'primary': True,
    'shortName': 'Hotel'}],
  'id': '4adcda33f964a520d33a21e3',
  'location': {'address': 'P. de la Castellana, 259',
   'cc': 'ES',
   'city': 'Madrid',
   'country': 'España',
   'distance': 282,
   'formattedAddress': ['P. de la Castellana, 259',
    '28046 Madrid Madrid',
    'España'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.476616494467,
     'lng': -3.6878589892993188}],
   'lat': 40.476616494467,
   'lng': -3.6878589892993188,
   'postalCode': '28046',
   'state': 'Madrid'},
  'name': 'Hotel Eurostars Madrid Tower',
  'photos': {'count': 0, 

In [72]:
dataframe_vir = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe_vir.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_vir_filtered = dataframe_vir.loc[:, filtered_columns]

# filter the category for each row
dataframe_vir_filtered['venue.categories'] = dataframe_vir_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_vir_filtered.columns = [col.split('.')[-1] for col in dataframe_vir_filtered.columns]

dataframe_vir_filtered.head(30)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hotel Eurostars Madrid Tower,Hotel,"P. de la Castellana, 259",ES,Madrid,España,NaN,282,"[P. de la Castellana, 259, 28046 Madrid Madrid...","[{'label': 'display', 'lng': -3.68785898929931...",40.476616,-3.687859,NaN,28046,Madrid,4adcda33f964a520d33a21e3
1,Welow,Restaurant,"Paseo de La Castellana, 259",ES,Madrid,España,NaN,194,"[Paseo de La Castellana, 259, 28046 Madrid Mad...","[{'label': 'display', 'lng': -3.68758365892808...",40.477374,-3.687584,NaN,28046,Madrid,5464bcc1498e255d34285fb4
2,Starbucks,Coffee Shop,"P. de la Castellana, 278",ES,Madrid,España,NaN,347,"[P. de la Castellana, 278, 28046 Madrid Madrid...","[{'label': 'display', 'lng': -3.68667036186360...",40.475923,-3.686670,NaN,28046,Madrid,555194a6498eedc18f4fdabc
3,Hamburguesa Nostra,Restaurant,"Paseo de la Castellana, 257. Terraza",ES,Madrid,España,NaN,473,"[Paseo de la Castellana, 257. Terraza, 28046 M...","[{'label': 'display', 'lng': -3.68756037974170...",40.474813,-3.687560,NaN,28046,Madrid,50e40e0cd63eeb05a583fb68
4,Torre de Cristal,Building,"Paseo de la Castellana, 259",ES,Madrid,España,NaN,98,"[Paseo de la Castellana, 259, 28046 Madrid Mad...","[{'label': 'display', 'lng': -3.68722200393676...",40.478194,-3.687222,NaN,28046,Madrid,4c7385ff376da093d441a9c6
5,Restaurante Vaca Nostra,Spanish Restaurant,"P. de la Castellana, 257",ES,Madrid,España,NaN,476,"[P. de la Castellana, 257, 28046 Madrid Madrid...","[{'label': 'display', 'lng': -3.68761458264409...",40.474792,-3.687615,NaN,28046,Madrid,50f188e0e4b0ff867a0380d9
6,Mercadona,Supermarket,Mauricio Legendre,ES,Chamartín,España,José Vasconcelos,892,"[Mauricio Legendre (José Vasconcelos), Chamart...","[{'label': 'display', 'lng': -3.68674949753611...",40.471022,-3.686749,NaN,NaN,Madrid,5720e0c2498e6b924cdf2e6a
7,Casa Nemesio,Spanish Restaurant,"P. de la Castellana, 260",ES,Madrid,España,NaN,752,"[P. de la Castellana, 260, 28046 Madrid Madrid...","[{'label': 'display', 'lng': -3.68707946877824...",40.472279,-3.687079,NaN,28046,Madrid,4b6ad6d0f964a520cce22be3
8,Queen Food & Music,Spanish Restaurant,"C. San Modesto, 34",ES,Madrid,España,NaN,830,"[C. San Modesto, 34, 28034 Madrid Madrid, España]","[{'label': 'display', 'lng': -3.69190380785420...",40.485441,-3.691904,NaN,28034,Madrid,4c025292f56c2d7f0f401b66
9,AUTOCINE Madrid,Drive-in Theater,"C. Antonio de Cabezón, 1",ES,Madrid,España,NaN,1147,"[C. Antonio de Cabezón, 1, 28034 Madrid Madrid...","[{'label': 'display', 'lng': -3.67796863078634...",40.486814,-3.677969,NaN,28034,Madrid,58ac7d6ffebfbd2f7145d4f7


In [73]:
venues_map_vir = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around Ecco


# add Venue spot as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Spot',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map_vir)

# display map
venues_map_vir

In [1]:
address3 = 'Paseo de la Castellana 259D, Madrid, Spain'

geolocator3 = Nominatim(user_agent="foursquare_agent")
location3 = geolocator3.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print(latitude3, longitude3)


NameError: name 'Nominatim' is not defined